In [ ]:
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine
from sqlalchemy import text
import getpass  # To get the password without showing the input
password = getpass.getpass()

In [ ]:
sd = "sakila"
connection_string = 'mysql+pymysql://root:' + password + '@localhost/'+sd
engine = create_engine(connection_string)
engine

In [ ]:
def rentals_month(engine, month=5, year=2005):
    with engine.connect() as connection:
        txt = f'''select *
                    from rental
                    where month(rental_date)="{month:02d}%" and year(rental_date)="{year}";'''
        query = text(txt)
        result = connection.execute(query)
    return pd.DataFrame(result.all())

In [ ]:
rentals_month(engine, 5, 2005)

In [ ]:
def rental_count_month(df_base, month=5, year=2005):
    column_name = f'rentals_{month:02d}_{year}'
    df_new = df_base.groupby(['customer_id']).size().reset_index(name=column_name)
    return df_new

In [ ]:
rental_count_month(rentals_month(engine, 5, 2005), 5, 2005)

In [ ]:
def compare_rentals(df1, df2):
    df_compared = df1.merge(df2, on='customer_id', how='inner', suffixes=('_df1', '_df2'))
    df_compared['difference'] = df_compared.iloc[:, 1] - df_compared.iloc[:, 2]
    return df_compared

In [ ]:
compare_rentals(rental_count_month(rentals_month(engine, 5, 2005), 5, 2005), rental_count_month(rentals_month(engine, 6, 2005), 6, 2005))